#### Import Dependencies

In [1]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Document, Prefetch, FusionQuery, Filter, FieldCondition, MatchText
from qdrant_client import models

from langsmith import traceable, get_current_run_tree


from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.messages import convert_to_openai_messages, convert_to_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json

from utils.utils import  format_ai_message, get_tool_descriptions

/Users/camilo/retrieval_rec_amazon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Define Retrieval Tool

In [2]:
@traceable(
    name="embed_query",
    run_type="embedding",
    metadata={"ls_provider": "openai", "ls_model_name": "text-embedding-3-small"}
)
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    current_run = get_current_run_tree()

    if current_run:
        current_run.metadata["usage_metadata"] = {
            "input_tokens": response.usage.prompt_tokens,
            "total_tokens": response.usage.total_tokens,
        }

    return response.data[0].embedding


@traceable(
    name="retrieve_data",
    run_type="retriever"
)
def retrieve_data(query, k=5):

    query_embedding = get_embedding(query)
    qdrant_client = QdrantClient(url="http://qdrant:6333")
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-02-hybrid-search",
        prefetch = [
            Prefetch(
                query= query_embedding,
                using = "text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using = "bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


@traceable(
    name="format_retrieved_context",
    run_type="prompt"
)
def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context

def get_formatted_context(query:str, top_k: int = 5)-> str:
    """Get the top k contexts, each representing an inventory item for a given query.
    
    Args:
        query: The query to get the top k contexts from
        top_k: The number of context chunks to retrieve, works best with 5 or more
        
    Returns:
        A string of the top k context chunks with IDs and average ratings prepending each chunk, each representing an inventory item for a given query."""

    context = retrieve_data(query, top_k)
    formatted_context = process_context(context)

    return formatted_context

#### State and Pydantic Models for Structured Outputs

In [13]:
class ToolCall(BaseModel):
    name: str
    arguments: dict

class RAGUsedContext(BaseModel):
    id: str = Field(description =" The ID of the item used to answer the question")
    description: str = Field(description =" Short description of the item used to answer the question")

class State(BaseModel):
    messages: Annotated[List[str], add] = []
    question_relevant: bool = False
    iteration: int = 0
    answer: str = ""
    available_tools : List[Dict[str, Any]] = []
    tool_calls: List[ToolCall] = []
    final_answer: bool = False
    references : Annotated[List[RAGUsedContext], add] = []

In [ ]:

@traceable(
    name = "agent_node",
    run_type = "llm",
    metadata = {"ls_provider": "openai", "ls_model_name": "gpt-4.1-mini"}
)
def agent_node(state:State) -> dict:
    prompt_template = """"You are a shopping assistant that can answer questions about the products in stock.

You will be given a conversation history and a list of tools you can use to answer the latest query.

<Available tools>
{{ available_tools | tojson }}
</Available tools>

When making tool calls, use this exact format:
{
    "name": "tool_name",
    "arguments": {
        "parameter1": "value1",
        "parameter2": "value2",
    }
}

CRITICAL: All parameters must go inside the "arguments" object, not at the top level of the tool call.

Examples:
- Get formatted item context:
{
    "name": "get_formatted_item_context",
    "arguments": {
        "query": "Kool kids toys.",
        "top_k": 5
    }
}

CRITICAL RULES:
- If tool_calls has values, final_answer MUST be false
(You cannot call tools and exit the graph in the same response)
- If final_answer is true, tool_calls MUST be []
(You must wait for tool results before exiting the graph)
- If you need tool results before answering, set:
tool_calls=[...], final_answer=false
- After receiving tool results, you can then set:
tool_calls=[], final_answer=true
- Use names specificly provided in the available tools. Don't add any additional text to the names.

Instructions:
- You need to answer the question based on the outputs from the tools using the available tools only.
- Do not suggest the same tool call more than once.
- If the question can be decomposed into multiple sub-questions, suggest all of them.
- If multipple tool calls can be used at once to answer the question, suggest all of them.
- Do not explain your next steps in the answer, instead use tools to answer the question.
- Never use word context and refer to it as the available products.
- You should only answer questions about the products in stock. If the question is not about the products in stock, you should ask for clarification.
- As an output you need to return the following:

* answer: The answer to the question based on your current knowledge and the tool results.
* references: The list of the indexes from the chunks returned from all tool calls that were used to answer the question. If more than one chunk was used to compile the answer from a single tool call, be sure to return all of them.
* Each reference should have an id and a short description of the item based on the retrieved context.
* final_answer: True if you have all the information needed to provide a complete answer, False otherwise.

- The answer to the question should contain detailed information about the product and should be returned with detailed specification in bullet points.
- The short description should have the name of the item.
- If the user's request requires using a tool, set tool_calls with the appropriate function names and arguments.
"""
    
    template = Template(prompt_template)

    prompt = template.render(
        available_tools = state.available_tools
    )
    messages = state.messages
    conversation = convert_to_openai_messages(messages)

    client = instructor.from_openai(OpenAI())

    response, raw_response = client.chat.completions.create_with_completion(
        model ="gpt-4.1-mini",
        response_model = AgentResponse,
        messages= [{"role": "system", "content": prompt}, *conversation],
        temperature =0.5,
    )

    ai_message = format_ai_message(response)

    return {
        "messages": [ai_message],
        "tool_calls": response.tool_calls,
        "iteration": state.iteration +1,
        "answer": response.answer
    }